# Sentence Dependencies

Ok, so a big part in getting better scores will be to narrow down the problem of the sentence pairs, for this we will need to know important words and their dependents. This notebook will serve solely to work out methods how to get those, so the other ones are not overloaded with stuff (like First Steps).

In [91]:
import spacy
import numpy as np
import pandas as pd

parser = spacy.load('en_core_web_sm')

test_a = 'I put a elephant in the fridge'
test_b = 'I put a turkey in the fridge'

# for this we want to remove stop words, to be implemented
different_words = ['elephant', 'turkey']
different_words_idx = [3,4]

parse_a = parser(test_a)
parse_b = parser(test_b)
print(type(parse_a))
print(parse_b)
for token in parse_a:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])
print('\n\n')
for token in parse_b:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

<class 'spacy.tokens.doc.Doc'>
I put a turkey in the fridge
I nsubj put VERB []
put ROOT put VERB [I, elephant, in]
a det elephant NOUN []
elephant dobj put VERB [a]
in prep put VERB [fridge]
the det fridge NOUN []
fridge pobj in ADP [the]



I nsubj put VERB []
put ROOT put VERB [I, turkey, in]
a det turkey NOUN []
turkey dobj put VERB [a]
in prep put VERB [fridge]
the det fridge NOUN []
fridge pobj in ADP [the]


In [120]:
""" THis will be methof get_dependents"""

"""PYTHONIC: maybe it would be cleaner to only do one sentence at a time,
and handle the structure of the sentences further up in the logic"""

#print(type(parse_a))
def get_dependents(parser, different_words, different_words_idx, sen):
    parse_a = parser(sen)
    #print(parse_a, different_words)
    #print(parse_a)
    #for w, idx in zip(different_words, different_words_idx):
    for w in different_words.split():
        w = parser(w).text
        if w == ' ':
            continue
        idx = get_dependent_index(parse_a, w)
        if idx == None:
            continue
        if w in parse_a.text:
            parent = parse_a[idx].head
            #print(parent)

            # first we check what pos tags our parent has, in order to build a decision tree...
            if parent.pos_ == 'VERB':
                # now we want all children of the verb with subj and obj labels
                verb_dependents_a = []
                for child in parent.children:
                    if child.dep_[-2:] == 'bj':
                        verb_dependents_a.append(child)
                    else:
                        for nephew in child.children:
                            if nephew.dep_[-2:] == 'bj':
                                verb_dependents_a.append(nephew)
    
    try:
        return verb_dependents_a
    except: return 0
    """                    
    elif w in parse_b.text:
        parent = parse_b[idx].head
        
        # first we check what pos tags our parent has, in order to build a decision tree...
        if parent.pos_ == 'VERB':
            # now we want all children of the verb with subj and obj labels
            verb_dependents_b = []
            for child in parent.children:
                if child.dep_[-2:] == 'bj':
                    verb_dependents_b.append(child)
                else:
                    for nephew in child.children:
                        if nephew.dep_[-2:] == 'bj':
                            verb_dependents_b.append(nephew)
                            
    """

In [ ]:
for i, w in enumerate(parse_a):
    

In [34]:
print(verb_dependents_a)
print(verb_dependents_b)

[I, elephant, fridge]
[I, turkey, fridge]


In [4]:
"""This will be method compute_similarity"""
#similarityy of two tokens ex:
# apples_oranges = apples.similarity(oranges)
#dependents = verb_dependents_a
def compute_similarity(dependents):
    similarity = 0
    cur, prev = None, None
    for w in dependents:
        cur = w
        #print(cur)
        if prev == None:
            prev = cur
            continue
        #print(prev)
        similarity += prev.similarity(cur)
        #print(similarity)
        prev = cur
    return similarity
    

In [40]:
similarity_a = similarity
similarity_a

0.49096861109137535

In [42]:
similarity_b = similarity
similarity_b

0.619660884141922

In [117]:
#def get_dependent_index(parser, sent, word):
def get_dependent_index(sent, word):
    
    #parse = parser(sent)
    #print(type(sent))
    #print('inside get_dependent_idx:', word)
    idx = 0
    for i, token in enumerate(sent):
        #print( token.text)
        if word == token.text:
            return i


In [94]:
# main
import spacy
import numpy as np
import pandas as pd


url_data_task_A = "https://raw.githubusercontent.com/wangcunxiang/SemEval2020-Task4-Commonsense-Validation-and-Explanation/master/Training%20%20Data/subtaskA_data_all.csv"
data_task_A = pd.read_csv(url_data_task_A,header=0, index_col=0)
#data_task_A = np.asarray(data_task_A)
different_words = pd.read_csv('different_words.csv', index_col=0)
#different_words = np.asarray(different_words)
different_words_idx = np.load('idx_different_words.npy')




In [68]:
different_words[:4]

,0,1
0,orange juice,milk
1,apple,milk
2,a mile,"100 , 000 miles"
3,##s me,i


In [133]:
import warnings
warnings.filterwarnings("ignore")

parser = spacy.load('en_core_web_sm')

for tuple_1, tuple_2 in zip(data_task_A.iterrows(),different_words.iterrows()):
    sen_0 = tuple_1[1]['sent0']
    sen_1 = tuple_1[1]['sent1']
    
    dif_0 = tuple_2[1][0]
    dif_1 = tuple_2[1][1]
    
    #print(dif_w_idx_0, dif_w_idx_1)
    dif_w_idx_0 = different_words_idx[tuple_1[0]][0]
    dif_w_idx_1 = different_words_idx[tuple_1[0]][1]
    #print(sen_0,dif_0,dif_w_idx_0,dif_w_idx_1)
    
    dependents_0 = get_dependents(parser, dif_0, dif_w_idx_0, sen_0)
    dependents_1 = get_dependents(parser, dif_1, dif_w_idx_1, sen_1)
    
    if not type(dependents_0) == int:
        #print(type(dependents_0))
        if len(dependents_0) > 0:
            sim_0 = compute_similarity(dependents_0)
            sim_1 = compute_similarity(dependents_1)
    
    print(sen_0,sen_1)
    print(dependents_0, dependents_1)
    print(sim_0, sim_1)
    if tuple_1[0] == 10: break

He poured orange juice on his cereal. He poured milk on his cereal.
[He, juice, cereal] [He, milk, cereal]
0.6528310328722 0.6681381613016129
He drinks apple. He drinks milk.
[He, apple] [He, milk]
0.1234443187713623 0.1834234744310379
Jeff ran a mile today Jeff ran 100,000 miles today
[Jeff, mile] [Jeff]
0.18877871334552765 0
A mosquito stings me I sting a mosquito
0 0
0.18877871334552765 0
A niece is a person. A giraffe is a person.
0 0
0.18877871334552765 0
A walk-in closet is larger than a normal closet. A normal closet is larger than a walk-in closet.
0 []
0.18877871334552765 0
I like to ride my chocolate I like to ride my bike
[chocolate] [bike]
0 0
A GIRL WON THE RACE WITH HER FRIEND A GIRL WON THE RACE WITH HORSE
0 0
0 0
he put elephant into the jug he pour water in to the jug
[he, elephant, jug] [he, water, jug]
0.44731343537569046 0.27087930403649807
A girl plays volleyball A dog plays volleyball
[girl, volleyball] [dog, volleyball]
0.25031012296676636 0.39596953988075256
Egg

In [116]:
dependents_1

0